# Определение стоимости автомобилей

### Описние проекта

**Цель проекта:**
Построить модель определения рыночной стоимости автомобилей для сервиса по продаже автомобилей с пробегом.

Заказчику важны:
- качество предсказания;
- скорость предсказания;
- время обучения;
- значение метрики RMSE должно быть меньше 2500.


**Исходные данные:**
Исторические данные сервиса по продаже автомобилей с пробегом: технические характеристики, комплектации и цены автомобилей.

**Описание данных:**

*Признаки*
* `DateCrawled` — дата скачивания анкеты из базы
* `VehicleType` — тип автомобильного кузова
* `RegistrationYear` — год регистрации автомобиля
* `Gearbox` — тип коробки передач
* `Power` — мощность (л. с.)
* `Model` — модель автомобиля
* `Kilometer` — пробег (км)
* `RegistrationMonth` — месяц регистрации автомобиля
* `FuelType` — тип топлива
* `Brand` — марка автомобиля
* `NotRepaired` — была машина в ремонте или нет
* `DateCreated` — дата создания анкеты
* `NumberOfPictures` — количество фотографий автомобиля
* `PostalCode` — почтовый индекс владельца анкеты (пользователя)
* `LastSeen` — дата последней активности пользователя

*Целевой признак*
* `Price` — цена (евро)

### Структура проекта

1. Подготовка данных
2. Обучение моделей
3. Итоговый вывод

## 1. Подготовка данных

In [1]:
# импортируем необходимые библиотеки
import os
import pandas as pd
import category_encoders as ce

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [2]:
# загружаем датасет
if os.path.exists(f'C:\\Users\\Anna\\Desktop\\Python\\Yandex Praktikum\\Datasets\\project_11\\autos.csv'):
    df = pd.read_csv(f'C:\\Users\\Anna\\Desktop\\Python\\Yandex Praktikum\\Datasets\\project_11\\autos.csv')
elif os.path.exists(f'/datasets/autos.csv'):
    df = pd.read_csv(f'/datasets/autos.csv')
else:
    print(f'Something is wrong')

In [3]:
# выводим таблицу, инфо о ней, а также количество явных дубликатов
display(df)
df.info()
print()
print(f'Количество явных дубликатов:{df.duplicated().sum()}')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
# сохраним изначальное количество строк датасета
start_size = df.shape[0]

# удаляем явные дубликаты
df = df.drop_duplicates()

In [5]:
# проверим уникальные значения столбца 'NumberOfPictures', вероятно он состоит из нулей
df['NumberOfPictures'].unique()

array([0], dtype=int64)

In [6]:
# удаляем столбцы, которые не дадут нам полезной информации для предсказания цены
df.drop(['DateCrawled', 'LastSeen', 'DateCreated', 'PostalCode', 'NumberOfPictures'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354365 non-null  int64 
 1   VehicleType        316875 non-null  object
 2   RegistrationYear   354365 non-null  int64 
 3   Gearbox            334532 non-null  object
 4   Power              354365 non-null  int64 
 5   Model              334660 non-null  object
 6   Kilometer          354365 non-null  int64 
 7   RegistrationMonth  354365 non-null  int64 
 8   FuelType           321470 non-null  object
 9   Brand              354365 non-null  object
 10  NotRepaired        283211 non-null  object
dtypes: int64(5), object(6)
memory usage: 32.4+ MB


In [7]:
# оставим объекты с правдоподобной стоимостью автомобиля от 100 евро (по анализу различных сайтов европы)
df = df.query('Price >= 100')

In [8]:
# посмотрим на описание значений столбца 'RegistrationYear'
df['RegistrationYear'].describe()

count    341051.000000
mean       2003.955335
std          69.812151
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

In [9]:
# оставим объекты с максимально правдоподобным годом выпуска от 1950 до 2016 (так как объявления датированы не позднее 2016 года)
df = df.query('RegistrationYear >= 1980 and RegistrationYear <= 2016')

In [10]:
# посмотрим на описание значений столбца 'Power'
df['Power'].describe()

count    324299.000000
mean        112.644566
std         183.800041
min           0.000000
25%          73.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64

In [11]:
# оставим объекты с максимально правдоподобным количеством лошадиных сил
df = df.query('Power >= 50 and Power <= 400')

In [12]:
# проверим дубликаты по колонке 'Model'
df['Model'].unique()

array([nan, 'grand', 'golf', 'fabia', '3er', '2_reihe', 'c_max',
       '3_reihe', 'passat', 'navara', 'polo', 'twingo', 'a_klasse',
       'scirocco', '5er', 'arosa', 'other', 'civic', 'transporter',
       'punto', 'e_klasse', 'clio', 'kadett', 'one', 'fortwo', '1er',
       'b_klasse', 'a8', 'jetta', 'fiesta', 'c_klasse', 'micra', 'vito',
       'sprinter', 'astra', '156', 'escort', 'forester', 'xc_reihe',
       'scenic', 'ka', 'a1', 'focus', 'a4', 'tt', 'a6', 'jazz', 'omega',
       'slk', '7er', 'combo', 'corsa', '80', '147', 'z_reihe', 'sorento',
       'ibiza', 'mustang', 'eos', 'touran', 'getz', 'insignia', 'almera',
       'megane', 'a3', 'r19', 'caddy', 'mondeo', 'cordoba', 'colt',
       'impreza', 'vectra', 'lupo', 'berlingo', 'tiguan', '6_reihe', 'c4',
       'panda', 'up', 'i_reihe', 'ceed', 'kangoo', '5_reihe', 'yeti',
       'octavia', 'zafira', 'mii', 'rx_reihe', '6er', 'modus', 'fox',
       'matiz', 'beetle', 'rio', 'touareg', 'logan', 'spider', 'cuore',
       's_m

In [13]:
# заменим неявные дубликаты 'rangerover' на 'range_rover'
df.loc[df['Model'] == 'rangerover', 'Model'] = 'range_rover'

In [14]:
# пропуски по колонке 'Model' заменим на заглушку 'unknown'
df.loc[df['Model'].isna(), 'Model'] = 'unknown'

In [15]:
# пропуски по колонке 'NotRepaired' заменим на заглушку 'unknown'
df.loc[df['NotRepaired'].isna(), 'NotRepaired'] = 'unknown'

In [16]:
# проверим дубликаты по колонке 'VehicleType'
df['VehicleType'].unique()

array(['coupe', 'suv', 'small', 'sedan', 'convertible', 'bus', 'wagon',
       nan, 'other'], dtype=object)

In [17]:
# проверим дубликаты по колонке 'Gearbox'
df['Gearbox'].unique()

array(['manual', 'auto', nan], dtype=object)

In [18]:
# проверим дубликаты по колонке 'FuelType'
df['FuelType'].unique()

array(['gasoline', 'petrol', nan, 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

In [19]:
# заменим неявные дубликаты 'petrol' на 'gasoline'
df.loc[df['FuelType'] == 'petrol', 'FuelType'] = 'gasoline'

In [20]:
def fill_na_by_model(col):
    df['Name'] = df['Brand'] + ' ' + df['Model']
    names = [name for name in list(df[df[col].isna()]['Name'].unique()) if isinstance(name, str) == True]
    for name in names:
        df.loc[(df['Name']==name)&(df[col].isna()), col] = df[df['Name']==name][col].value_counts().index[0]
        
for col in ['VehicleType', 'Gearbox', 'FuelType']:
    fill_na_by_model(col)

In [21]:
df.reset_index(drop=True, inplace=True)
df.info()
print()
print(f'В ходе предобработки потеря данных составила: {1 - df.shape[0] / start_size:.2%}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287976 entries, 0 to 287975
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              287976 non-null  int64 
 1   VehicleType        287976 non-null  object
 2   RegistrationYear   287976 non-null  int64 
 3   Gearbox            287976 non-null  object
 4   Power              287976 non-null  int64 
 5   Model              287976 non-null  object
 6   Kilometer          287976 non-null  int64 
 7   RegistrationMonth  287976 non-null  int64 
 8   FuelType           287976 non-null  object
 9   Brand              287976 non-null  object
 10  NotRepaired        287976 non-null  object
 11  Name               287976 non-null  object
dtypes: int64(5), object(7)
memory usage: 26.4+ MB

В ходе предобработки потеря данных составила: 18.74%


**Выводы**
* Данные успешно загружены:
    * в таблице 354369 строк, 16 столбцов;
    * колонка `'Price'` - целевой признак;
    * обнаружены явные дублиакаты;
    * колонки имеют пропуски.
* Произведена предобработка данных:
    * удалены явные дубликаты;
    * заменены неявные дубликаты по колонке;
    * пропуски в категориальных признаки заменены заглушкой `'unknown'`
    * удалены колонки не несущие в себе полезной информации: 
        * `'DateCrawled'`, 
        * `'LastSeen'`, 
        * `'DateCreated'`, 
        * `'PostalCode'`, 
        * `'NumberOfPictures'`.
    * Потеря данных после предобработки составила 4.17%, что является приемлемым показателем.

## 2. Обучение моделей

In [22]:
# выделяем фичи и таргет
y = df['Price']
X = df.drop('Price', axis=1)

In [23]:
# делим выборку на обущающую и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

* Модель **CatBoostRegressor**

In [24]:
%%time

# определяем категориальные признаки
cat_features = list(df.select_dtypes(include='object').columns)

# инициируем модель
cat = CatBoostRegressor(cat_features=cat_features, verbose=False)

# выделяем параметры для поиска наилуших с помощью 'GridSearchCV'
params = {'random_state': [42],
          'learning_rate': [0.1],
          'iterations': [100],
          'depth': range(2, 5)}

# делаем поиск параметров по метрике RMSE
grid_search = GridSearchCV(estimator = cat, 
                           param_grid = params, 
                           cv = 5, 
#                            n_jobs = -1, 
                           scoring = 'neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

print(f'Параметры лучшей модели: {grid_search.best_params_}')
print(f'RMSE лучшей модели: {grid_search.best_score_}')

Параметры лучшей модели: {'depth': 4, 'iterations': 100, 'learning_rate': 0.1, 'random_state': 42}
RMSE лучшей модели: -1818.0586177657428
CPU times: total: 6min 33s
Wall time: 2min 12s


In [25]:
%%time

# обучаем модель с лучшими параметрами и проверяем ее на тестовой выборке
cat.set_params(**grid_search.best_params_)
cat.fit(X_train, y_train)
pred = cat.predict(X_test)
print(f'RMSE на тестовой выборке: {mean_squared_error(y_test, pred, squared=False)}')

RMSE на тестовой выборке: 1800.2528220199245
CPU times: total: 36.4 s
Wall time: 11.2 s


* RMSE на тестовой выборке **1800.25**, что уже удовлетворяет условию задачи
* Время на поиск лучшей модели **6 мин. 33 сек**.
* Время на обучение и предсказание модели  **11 сек**.

---
* Модель **LGBMRegressor**

In [26]:
cat_features = list(df.select_dtypes(include='object').columns)

In [27]:
# кодируем категориальные признаки с помощью LeaveOneOutEncoder
loo_enc = ce.LeaveOneOutEncoder(cols=cat_features)
loo_enc.fit(X_train, y_train)
X_train = loo_enc.transform(X_train)
X_test = loo_enc.transform(X_test)

In [28]:
# масштабируем признаки
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
%%time

# инициируем модель
lgbm = LGBMRegressor(objective="regression")

# выделяем параметры для поиска наилуших с помощью 'GridSearchCV'
params = {'max_depth': range(32, 36),
         'num_leaves': [1000, 1200],
         'learning_rate': [0.1], 
         'random_state': [42]}

# делаем поиск параметров по метрике RMSE
grid_search = GridSearchCV(estimator = lgbm, 
                           param_grid = params, 
                           cv = 5, 
#                            n_jobs = -1, 
                           scoring = 'neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

print(f'Параметры лучшей модели: {grid_search.best_params_}')
print(f'RMSE лучшей модели: {grid_search.best_score_}')

Параметры лучшей модели: {'learning_rate': 0.1, 'max_depth': 32, 'num_leaves': 1000, 'random_state': 42}
RMSE лучшей модели: -1479.0458888484927
CPU times: total: 27min 30s
Wall time: 7min 8s


In [30]:
%%time

# обучаем модель с лучшими параметрами и проверяем ее на тестовой выборке
lgbm.set_params(**grid_search.best_params_)
lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
print(f'RMSE на тестовой выборке: {mean_squared_error(y_test, pred, squared=False)}')

RMSE на тестовой выборке: 1448.1076350510339
CPU times: total: 41.7 s
Wall time: 10.8 s


* RMSE на тестовой выборке **1448.11**, что удовлетворяет условию задачи, а также лучше модели `CatBoostRegressor`
* Время на поиск лучшей модели **7 мин. 8 сек**.
* Время на обучение и предсказание модели **10.8 сек**.

---
* Модель **Линейная регрессия**

In [31]:
%%time

# инициируем модель, обучаем и проверяем ее на тестовой выборке
lr = LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
print(f'RMSE на тестовой выборке: {mean_squared_error(y_test, pred, squared=False)}')

RMSE на тестовой выборке: 2630.5490556330037
CPU times: total: 266 ms
Wall time: 216 ms


* RMSE на тестовой выборке **2630.55**, что не удовлетворяет условию задачи.
* Время на обучение и предсказание модели **216 мсек**.

---
Проверим модели на адекватность с помощью регрессора **DummyRegressor**

In [32]:
dummy_regr = DummyRegressor(strategy='mean')
dummy_regr.fit(X_train, y_train)
pred = dummy_regr.predict(X_test)
print(f'RMSE на тестовой выборке: {mean_squared_error(y_test, pred, squared=False)}')

RMSE на тестовой выборке: 4595.941163537954


**Вывод**
* Все модели прошли проверку на адекватность (RMSE DummyRegressor = **4595.94**).
* Наиболее точной моделью по метрике RMSE оказалась `LGBMRegressor` (**1448.11**)
* Наиболее быстрой в плане обучения также оказалась модель `LGBMRegressor` (**10.8 сек.**)
* Заказчику будет предложена модель `LGBMRegressor` со следующими параметрами:
    * `learning_rate`: 0.1, 
    * `max_depth`: 32, 
    * `num_leaves`: 1000

## 3. Итоговый вывод
1. Данные были успешно загружены. Была произведена предобработка данных:
    * удалены явные дубликаты;
    * заменены неявные дубликаты по колонке;
    * пропуски в категориальных признаки заменены заглушкой `'unknown'`
    * удалены колонки не несущие в себе полезной информации: 
        * `'DateCrawled'`, 
        * `'LastSeen'`, 
        * `'DateCreated'`, 
        * `'PostalCode'`, 
        * `'NumberOfPictures'`.
    * Потеря данных после предобработки составила 4.17%, что является приемлемым показателем.
2. Были обучены три модели - CatBoostRegressor, LGBMRegressor и Линейная регрессия.
    * Наиболее точной моделью по метрике RMSE оказалась `LGBMRegressor` (**1448.11**)
    * Наиболее быстрой в плане обучения также оказалась модель `LGBMRegressor` (**10.8 сек.**)
    * Заказчику будет предложена модель `LGBMRegressor` со следующими параметрами:
        * `learning_rate`: 0.1, 
        * `max_depth`: 32, 
        * `num_leaves`: 1000